In [95]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import numpy as np
import vmo.analysis as van
import vmo.generate as vge
import matplotlib.pyplot as plt
import sklearn.preprocessing as pre
import librosa, vmo
import IPython.display
%matplotlib inline

/anaconda3/envs/vmo/lib/python2.7/site-packages/librosa/__init__.py:35: FutureWarning: You are using librosa with Python 2. Please note that librosa 0.7 will be the last version to support Python 2, after which it will require Python 3 or later.
  FutureWarning)


## Load audio file 

In [2]:
target_file = '/Users/meier/lyresuperbe.aif'
playback_y, sr = librosa.load(target_file)

In [98]:
#IPython.display.Audio(data=playback_y, rate=sr)

## Parameters

In [3]:
sample_rate = 22050
fft_size = 2048*4
hop_size = fft_size/2

## Analysis

In [102]:
y, sr = librosa.load(target_file, sr=sample_rate)
C = librosa.feature.mfcc(y=y, sr=sr, S=None, n_mfcc=20, hop_length=hop_size)
feature = librosa.core.power_to_db(C)
feature = pre.normalize(feature)

In [104]:
mfcc_frames = feature.transpose()
r = (0.0, 1.01, 0.01) 
ideal_t = vmo.find_threshold(mfcc_frames, r = r,flag = 'a', dim=20)
oracle_t = vmo.build_oracle(mfcc_frames, flag = 'a', threshold = ideal_t[0][1], feature = 'mfcc', dim=20)

## Export/import Oracle

In [109]:
import pickle
pickle.dump(oracle_t, open(target_file+'_oracle.p', "wb"))

In [4]:
import pickle
oracle_t = pickle.load(open(target_file+'_oracle.p', "rb"))

## Plot

In [105]:
x = np.array([i[1] for i in ideal_t[1]])
y = [i[0] for i in ideal_t[1]]
fig = plt.figure(figsize = (10,3))
plt.plot(x, y, linewidth = 2)
plt.title('IR vs. Threshold Value(vmo)', fontsize = 18)
plt.grid(b = 'on')
plt.xlabel('Threshold', fontsize = 14)
plt.ylabel('IR', fontsize = 14)
plt.xlim(0,0.2)
plt.tight_layout()

## Synthesis

In [5]:
seq = vge.improvise(oracle_t, seq_len = oracle_t.n_states-1, LRS = 2, weight = 'lrs')
x, _w, new_sr = vge.audio_synthesis(target_file, input_file+'_vmo.wav', seq, analysis_sr=sample_rate, buffer_size = fft_size, hop = hop_size)

In [107]:
#IPython.display.Audio(data=x, rate=new_sr)